# Crash Course in Neural Networks with Keras
This is a brief introduction to using Keras, which will also teach you a bit about neural networks along the way.

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

ModuleNotFoundError: No module named 'keras'

## 0. Neural Network = Model
In broad terms machine learning is teaching a machine the relation between some input, $x$ and some output $y$, so 

$y = f(x, a) $

where $a$ is a set of parameters that define the function $f$.

A neural network can therefore be thought of as a model $f$ that has a set of parameters, $a$, that you try to optimize to best predict the relation between, a set of training images and their labels. 

Neural networks can also be used for regression problems, and very simplified analogy would be a very complicated, high-order polynomial. 

### Sequential models
Most neural networks, or models, you can construct with Keras fall under the sequential model type. This is the 'typical' network seen in most illustrations, and consists of a sequence of layers that each take inputs from neurons in the preceding layer, perform some operation depending on the layer type, and then feed it forward into the next layer in the sequence.

In 'classical' code, this would look something like:
```python
y = input(x)
y = layer1_func(y)
y = layer2_func(y)
y = output(y)
```

Keras gives access to many types of layers, corresponding to different arithmetic functions and array manipulations. Keras allows you to mix and match these layers in your sequence, and will automatically sort out the connections between them. 

In [9]:
# Initialising a sequential model
model = Sequential()

NameError: name 'Sequential' is not defined

## 1. Classification
This sets up a simple image classifier, to classify images of cats and dogs.

### Training data
To set up a training set, all you need to do it create a directory, where sub-directories correspond to each class. 

Example:

/home/training_set/cats

/home/training_set/dogs

Keras then only needs to be pointed at /home/training_set and it will figure out the classes, in this case cats and dogs.

In [10]:
# Directory where you have placed your training data
training_dir = '/home/training_set' # EDIT THIS
validation_dir = '/home/validation_set' # EDIT THIS

### Data Generators
Data generators are functions used for ingesting data and feeding it into your network, with some degree of manupulations along the way. They are typically used to feed in small batches of the training data, so as to conserve memory in case the training set is very large.

For image classification Keras has a built in class ImageDataGenerator that does this, and also preprocesses the images, and can perform data augmentation. 

#### Preprocessing
Image data comes in a variety of different shapes, sizes and formats. Your network will only function or at least function well, on images that are preprocessed in a similar fashion to what it's been trained on. It's therefore useful to establish a format specific for your network. These include, rescaling the pixel values to be between 0 and 255 (png standard), binning or interpolating the image to a specific shape. 

#### Data Augmentation
In general purpose classification it is sometimes useful to alter the training data in some way. This makes the network able to handle a greater variation in the unseen data. It can also be used to artificially increase the number of training examples.

In [ ]:
# Initialize the ImageDataGenerator class
train_datagen = ImageDataGenerator(rescale = 1./255, # preprocessing
                                   shear_range = 0.2, # data aug.
                                   zoom_range = 0.2, # data aug.
                                   horizontal_flip = True) # data aug. 

# Setup the ImageDataGenerator instance to ingest images from training_dir
training_set = train_datagen.flow_from_directory(training_dir,
                                                 target_size = (64, 64), # preprocessing
                                                 batch_size = 32, 
                                                 shuffle = True
                                                 class_mode = 'binary')

batchsize: number of images the generator yields each time it is called

shuffle: randomize the training set, so each batch doesn't accidentally contain instances of the same class

class_mode: specific to network type, classifiers can be 'binary' or 'categorical'

#### Validation data
The training data that the network never sees.

In [ ]:
# Initialize the ImageDataGenerator class
validation_datagen = ImageDataGenerator(rescale = 1./255)

# Setup the ImageDataGenerator instance to ingest images from training_dir
validation_set = test_datagen.flow_from_directory(validation_dir,
                                                  target_size = (64, 64),
                                                  batch_size = 32,
                                                  class_mode = 'binary')

### Building the network

In [ ]:
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Flatten())
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

In [ ]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the network

In [ ]:
classifier.fit_generator(training_set, 
                         steps_per_epoch = 8000, 
                         epochs = 25, 
                         validation_data = test_set,
                         validation_steps = 2000)

### Testing the network

In [ ]:
# Part 3 - Making new predictions
import numpy as np
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

## 2. Regression

## 3. Transfer Learning